## Descarga de los archivos

Descargaremos los archivos a disposicion en el siguiente link.

In [66]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-05-21 18:33:57--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-05-21 18:33:58 (750 MB/s) - ‘datasource.zip’ saved [4249/4249]



## Descompresion

descomprimiremos y guardaremos los archivos en la carpeta dealership_data:

Resultado: Tenemos varias archivos con diferentes extensiones, csv, json y xml

In [67]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


Ahora importaremos las librerias para el proceso:

In [68]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

## Rutas 

Establecemos archivos con los cuales podemos crear filas temporales, un archivo final con las transformaciones y el archivo historico con los acontecimientos del proceso

In [69]:
tmpfile = "temp.tmp" #archivo temporal para guardar todos los datos extraidos
logs = "log.txt" #historico del proceso
targetfile = "transformed_data.csv" #datos transformados

## Extraccion

In [70]:
def extract_from_csv(file):
    df = pd.read_csv(file)
    return df

In [71]:
def extract_from_json(file):
    df = pd.read_json(file, lines=True)
    return df

In [72]:
def extract_from_xml(file):
    df = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])
    tree = ET.parse(file)
    root  = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = int(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        df = df.append({"car_model": car_model, "year_of_manufacture": year_of_manufacture, "price": price, "fuel": fuel}, ignore_index=True)
    return df 

In [73]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])

    number_csv_files = 0
    for csvfile in glob.glob('./dealership_data/*.csv'):
        extracted_data = extracted_data.append(extract_from_csv(csvfile) , ignore_index=True)
        number_csv_files += 1


    for jsonfile in glob.glob('./dealership_data/*.json'):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    for xmlfile in glob.glob('./dealership_data/*.xml'):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
    
    return extracted_data

## Transformacion

Las transformaciones consisten en:

- Obtener el precio en 2 decimales de acuerdo a los estandares de precio:

In [74]:
def transform(data):
    data['price'] = data['price'].astype(float, errors='raise')
    data['price'] = round(data['price'], 2)
    return data


## Carga

Cargaremos los datos a un csv con todo el contenido:

In [75]:
def load(targetfile, transform_data):
    transform_data.to_csv(targetfile)

## Logs

Es la funcion que registrará en una archivo el historico del proceso de ETL

In [76]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logs.txt", "a") as f:
        f.write(timestamp + ' - ' + message + '\n')

## Main

Ejecucion de nuestro proceso de ETL

In [77]:
log("INICIO DEL PROCESO DE ETL")

In [78]:
log("Fase de extraccion de datos iniciada.")
extracted_data = extract()
log("Proceso de extraccion de datos finalizado.")
extracted_data

/tmp/ipykernel_3672/420101544.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile) , ignore_index=True)
/tmp/ipykernel_3672/420101544.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile) , ignore_index=True)
/tmp/ipykernel_3672/420101544.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile) , ignore_index=True)
/tmp/ipykernel_3672/420101544.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(

,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.731343,Petrol
1,ciaz,2015,10223.880597,Diesel
2,ciaz,2015,11194.029851,Petrol
3,ertiga,2015,9104.477612,Petrol
4,dzire,2009,3358.208955,Petrol
...,...,...,...,...
85,etios liva,2014,7089.552239,Diesel
86,innova,2017,29477.61194,Petrol
87,fortuner,2010,13805.970149,Diesel
88,corolla altis,2011,6492.537313,Petrol


In [79]:
extracted_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   car_model            90 non-null     object
 1   year_of_manufacture  90 non-null     object
 2   price                90 non-null     object
 3   fuel                 90 non-null     object
dtypes: object(4)
memory usage: 2.9+ KB


In [80]:
log("Fase de transformacion de datos iniciada.")
transformed_data = transform(extracted_data)
log("Proceso de transformacion de datos finalizado.")
transformed_data

,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.73,Petrol
1,ciaz,2015,10223.88,Diesel
2,ciaz,2015,11194.03,Petrol
3,ertiga,2015,9104.48,Petrol
4,dzire,2009,3358.21,Petrol
...,...,...,...,...
85,etios liva,2014,7089.55,Diesel
86,innova,2017,29477.61,Petrol
87,fortuner,2010,13805.97,Diesel
88,corolla altis,2011,6492.54,Petrol


In [81]:
transformed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   car_model            90 non-null     object 
 1   year_of_manufacture  90 non-null     object 
 2   price                90 non-null     float64
 3   fuel                 90 non-null     object 
dtypes: float64(1), object(3)
memory usage: 2.9+ KB


In [82]:
log("Fase de carga de datos iniciada.")
load(targetfile, transformed_data)
log("Fase de carga de los datos finalizada.")

In [83]:
log("PROCESO FINALIZADO CON EXITO")

In [84]:
#try:   
#    !jupyter nbconvert --to python .ipynb
    # Python se convierte a .py, el script se convierte a .html
         # file_name.ipynb es el nombre del archivo del módulo actual
#except:
#    pass